## Real data

Die Daten im Notebook 03_data.ipynb habe ich für euch vorbereitet. 
Leider begegnen uns Daten in den meisten Fällen nicht so schön maschinenlesbar. 
Die Wahrheit ist: Da draußen ist der Daten-Dschungel und es ist gar nicht so einfach das, was wir brauchen, da rauszuziehen. 

**So findest du die bittere Wahrheit heraus** 
- Gehe auf [https://www-genesis.destatis.de/] 
- suche nach 'Durchschnittsalter der habilitierten Personen'. 
- Dann auf 'Werteabruf' und die Tabelle bitte als 'Flat' in ./data abspeichern.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_rows = 999

In [ ]:
df = pd.read_csv('./data/21351-0002_flat.csv')

### Huppla - was ist passiert?

Schau dir die Tabelle in einem Tabellenprogramm an.
Was fällt dir auf?

- In deutschen Tabellen wird ';' statt ',' als Trenner für .csvs verwendet
- Dafür schreiben wir in Deutschland '42,3' statt '42.3'
- In der Tabelle gibt es fehlende Werte, die mit '-'
- Oft wird auch nicht das weltweit gebräuchliche 'uft-8' als Zeichenformat verwendet, sondern 'latin-1' oder 'iso-8859-1'

In [ ]:
df = pd.read_csv('./data/21351-0002_flat.csv', sep=';', decimal=',', encoding='latin-1')
# sep steht für seperator, also Trennzeichen
# decimal soll '42,3' automatisch in '42.3' umwandeln
# encoding gibt das Zeichenformat an

In [ ]:
df.head(20)

### Puh, das sind viele Spalten!

In [ ]:
# Zeige alle Spalten an 
df.columns

In [ ]:
# So wählst du aus, welche Spalten du behalten willst
df = df[['Statistik_Label', 
         'Zeit', 
         '2_Auspraegung_Label',
         '3_Auspraegung_Label',
         'BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre']]

In [ ]:
df.head()

In [ ]:
# So kannst du ausgewählte Spalten weglassen 
df.drop(['Statistik_Label'], axis='columns', inplace=True)

In [ ]:
# So sieht die Tabelle schon übersichtlicher aus
df.sample(5)

## Daten filtern

Wir müssen auswählen, was wir als in der Grafik darstellen wollen. Sagen wir mal uns interessiert besonders, wie sich das Alter bei der Habilitation im Bereich Kunst- und Kunstgeschichte Insgesamt verändert hat:

In [ ]:
# So wählst du den DataFrame aus, wo die Spalte "2_Ausprägung_Label" den Wert 'Kunst, Kunstgeschichte' enthält 
df[(df["2_Auspraegung_Label"] == 'Kunst, Kunstwissenschaft')]

In [ ]:
# Wenn du nur die Daten für 'Insgesamt' auswählen willst
df[(df["2_Auspraegung_Label"] == 'Kunst, Kunstwissenschaft') & (df["3_Auspraegung_Label"] == 'Insgesamt')]

In [ ]:
# Wenn du für Kunstgeschichte die Daten für 'männlich' oder 'weiblich' auswählen willst
df[(df["2_Auspraegung_Label"] == 'Kunst, Kunstwissenschaft') & ((df["3_Auspraegung_Label"] == 'weiblich') | (df["3_Auspraegung_Label"] =='männlich')) ]

In [ ]:
# Wenn du dir sicher bist, dass das richtige gefiltert wird,kannst du einen neuen DataFrame erzeugen. So bleiben die Ursprungsdaten erhalten.
kunst = df[(df["2_Auspraegung_Label"] == 'Kunst, Kunstwissenschaft') & ((df["3_Auspraegung_Label"] == 'weiblich') | (df["3_Auspraegung_Label"] =='männlich')) ]

In [ ]:
kunst

## Visualisierung

In [ ]:
kunst

In [ ]:
kunst = kunst.pivot(index='Zeit', columns='3_Auspraegung_Label', values='BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre')

In [ ]:
kunst

In [ ]:
kunst.plot()

### Hm, das mit dem Zahlenformat hat nicht geklappt

In [ ]:
# In der Spalte mit dem Durchschnittsalter ist das Komma leider nicht korrekt ersetzt worden. Shit happens.
# Eine einzelne Spalte kann mit df.Name_der_Spalte aufgerufen werden. (Falls der Name der Spalte Leerzeichen oder Sonderzeichen enthält mit df['Name der Spalte'])
kunst.männlich

In [ ]:
# Dass das nicht geklappt hat, liegt daran, dass es ürsprünglich mal fehlende Werte gab, die das Zeichen '-' enthalten haben (Bei 'Veterinärmedizin' gibt es in der .csv von destatis keine Daten ab 2015)
# Durch das Filtern sind wir diese seltsamen Werte losgeworden. Leider enthalten unsere Werte immer noch ',' statt '.'.
# Um zu sehen, welche Werte vorhanden sind, kannst du zum Beispiel zählen lassen, wie welche Werte wie oft vorkommen.   
kunst.weiblich.value_counts()

In [ ]:
# Also bereinigen wir die Spalte selbst. Pandas ist gut darin, Strings zu verändern. In diesem Fall wollen wir in unserer Spalte ',' durch '.' ersetzen
kunst.männlich = kunst.männlich.str.replace(',', '.')
kunst.weiblich = kunst.weiblich.str.replace(',', '.')

In [ ]:
kunst.head()

In [ ]:
# Die ',' wurden zwar durch '.' ersetzt, aber der Datentyp ist immer noch 'Object'. Wir brauchen aber numerische Daten (int, float) zum plotten. 
kunst.dtypes

In [ ]:
kunst.plot()

In [ ]:
# So änderst du den Datentyp auf float
kunst.männlich = kunst.männlich.astype('float')
kunst.weiblich = kunst.weiblich.astype('float')

In [ ]:
kunst

In [ ]:
kunst.plot()

In [ ]:
kunst.plot(kind="bar", figsize=(15,6))

## Übung

In [ ]:
# Zeige, wie sich das Durchschnittsalter der habilitierten Personen in Deutschland insgesamt verändert hat


In [ ]:
# Stelle die Veränderung in einem Linechart dar


## Gruppieren

### In welchem Bereich ist das Habilitationsalter am höchsten?

Um Durchschnitte berechnen zu können, muss die Werte Spalte den Datentyp 'float' haben, also ausschließlich Nummern enthalten.

In [ ]:
df.dtypes

Um die Spalte zu bereinigen:

- müssen wir den kompletten Datensatz von fehlendenden Werten befreien
- müssen wir die Spalte in numerische Werte umwandeln 

In [ ]:
# Durch value_counts sehen wir, dass häufig '-' bei fehlenden Daten auftaucht.
df['BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre'].value_counts().head()

In [ ]:
# Anzeigen, in welchen Fällen das so ist:
df[df.BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre == '-']

In [ ]:
# Die entsprechenden Fachrichtungen aussortieren
df = df[(df['2_Auspraegung_Label'] != 'Veterinärmedizin') & 
        (df['2_Auspraegung_Label'] != 'Agrar-, Forst- und Ernährungswissenschaften') & 
        (df['2_Auspraegung_Label'] != 'Agrar-, Forst- u.Ernährungswiss., Veterinärmedizin') &
        (df['2_Auspraegung_Label'] != 'Sport')
       ]

In [ ]:
# ',' Werte durch '.' ersetzen
df.BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre = df.BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre.str.replace(',', '.')

In [ ]:
# Datentyp auf float ändern
df.BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre = df.BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre.astype('float')

In [ ]:
# So wird der Durchschnitt der Jahre gebildet
alter_im_schnitt = df.groupby('2_Auspraegung_Label').mean().round(2).sort_values('BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre', ascending=False)
alter_im_schnitt

Hier noch mal Schritt für Schitt erklärt:

Um das Alter im Schnitt zu berechnen
- gruppieren wir nach '2_Ausprägung_Label': .groupby('2_Auspraegung_Label')
- nehmen jeweils den Durchschnitt: .mean()
- runden das Ergebnis auf 2 Stellen: .round(2)
- sortieren die Werte absteigend: .sort_values('BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre', ascending=False)

In [ ]:
# Daten plotten
alter_im_schnitt.plot.barh(y='BIL051__Durchschnittsalter_der_habilitierten_Personen__Jahre', 
                                           legend=False, 
                                           title="Durchschnittsalter der habilitierten Personen",
                                          figsize=(12,10))

## Übung

Aus welchem Gebiet stammen im Schnitt die jüngsten männlichen Eingebürgerten?

- Suche auf [https://www-genesis.destatis.de/genesis/online] nach 'Durchschnittsalter der Eingebürgerten: Deutschland, Jahre, Ländergruppierungen/Staatsangehörigkeit, Geschlecht,Familienstand
- Lade den Datensatz als Flat.csv herunter
- Lesen ihn als DataFrame ein
- Wähle folgende Spalten aus: '2_Auspraegung_Label', '3_Auspraegung_Label', '4_Auspraegung_Label', 'BEV064__Durchschnittsalter_der_Eingebuergerten__Jahre'
- Filtere auf 'männlich' und 'ledig
- Sortiere den Datensatz nach dem Durchschnittsalter 

## Übung

- Such dir einen Datensatz in [https://www-genesis.destatis.de/]
- Lade den Datensatz als Flat-Table herunter
- Lese ihn als DataFrame ein
- Stelle dir eine Frage zu diesem Datensatz, die du beantworten willst 
- Wähle die Spalten aus, die du brauchst
- Filtere den Datensatz ggf. so, dass fehlende Daten ausgeblendet werden
- Gruppiere oder sortiere falls nötig
- Beantworte deine Frage

Hervorragend! 

Weiter geht es mit dem 

**[>> Datenportal Notebook](04b_real_real_data_datenportal_csv.ipynb)**